# Importaciones

In [157]:
import pandas as pd
import os

Creación de directorios si no existen

In [158]:
os.mkdir("Datawarehouse") if not os.path.exists("Datawarehouse") else None

# Carga de dataframes

Se realiza una carga de los dataframes exportados previamente, para acelerar el proceso de testeo, ya que los pasos anteriores requieren de un alto consumo de tiempo.

In [159]:
df_restaurantes = pd.read_parquet(r'Datalake\restaurantes.parquet')
df_reviews = pd.read_parquet(r'Datalake\reviews.parquet')
df_estados = pd.read_parquet(r'Datalake\estados.parquet')

## Eliminación de datos duplicados según ID y restaurantes sin id_estado

In [160]:
df_restaurantes = df_restaurantes.drop_duplicates(subset=["id_restaurante"], keep='last')
df_reviews = df_reviews.drop_duplicates(subset=["id_resena"], keep='last')
df_restaurantes = df_restaurantes.dropna(subset=['id_estado'])

# Conversión de datos por columna

In [161]:
df_restaurantes.loc[:, 'cantidad_resenas'] = df_restaurantes['cantidad_resenas'].astype('uint32')
df_restaurantes.loc[:, 'calificacion'] = df_restaurantes['calificacion'].astype('float16')
df_restaurantes.loc[:, 'id_estado'] = df_restaurantes['id_estado'].astype('uint8')
df_reviews.loc[:, 'calificacion'] = df_reviews['calificacion'].astype('float16')
df_reviews.loc[:, 'anio'] = df_reviews['anio'].astype('float16')
df_reviews.loc[:, 'puntaje_de_sentimiento'] = df_reviews['puntaje_de_sentimiento'].astype('float16')
df_estados.loc[:, 'id_estado'] = df_estados['id_estado'].astype('uint8')
df_estados.loc[:, 'Population'] = df_estados['Population'].astype('uint32')

# Ejecución de análisis de sentimiento

Los elementos sin procesar tienen un puntaje de sentimiento = 100, los que serán almacenados temporalmente en df_analisis para ser procesados y luego se reemplazarán los valores correspondientes en df_reviews

In [162]:
df_reviews.head()

,id_usuario,id_restaurante,id_resena,calificacion,anio,puntaje_de_sentimiento,texto
0,RreNy--tOmXMl1en0wiBOg,cPepkJeRMtHapc_b2Oe_dw,lUUhg8ltDsUZ9h0xnwY4Dg,4,2018,0,None
1,OuatwND396ZQxm2zK8WlUQ,jNL5KUPz2-tHUJM__ysSaw,TcCcHzc3L6Aboq3DteEfZA,1,2014,0,None
5,07aVWNhBhpwqW3MdqoKDKQ,jLerIZFpe25jOUf722qzBA,zRIH5eDA2GbynjZPrk8dDg,3,2011,0,None
9,Nsm9V1s8JQyuYsShtiwplg,eUO4QA6nL2XdCeWuD7tkww,UrGHZY5-pDs4uOP8SFPg-w,5,2017,0,None
16,5C_C7bd6rZYgiP4puQxESQ,0Kn5W22UmxOqPj2cjouFNA,Zb_27vX8weaYyDn-_2ZhVA,1,2015,0,None


In [163]:
#extracción de líneas a analizar
df_analisis = df_reviews[df_reviews['puntaje_de_sentimiento'] == 100].copy()

## Análisis de sentimiento

In [164]:
# acá tiene que ir e análisis de sentimiento, elimina éste código inútil de prueba
df_analisis['puntaje_de_sentimiento'] = 100
df_analisis.head(1)

,id_usuario,id_restaurante,id_resena,calificacion,anio,puntaje_de_sentimiento,texto
0,RreNy--tOmXMl1en0wiBOg,cPepkJeRMtHapc_b2Oe_dw,lUUhg8ltDsUZ9h0xnwY4Dg,4,2018,100,None


## Actualización de datos post-análisis

In [165]:
df_reviews.loc[df_reviews['id_resena'].isin(df_analisis['id_resena']), 'puntaje_de_sentimiento'] = df_analisis['puntaje_de_sentimiento']

In [166]:
df_restaurantes.to_parquet(r"Datawarehouse\restaurantes.parquet")
df_reviews.to_parquet(r"Datawarehouse\reviews.parquet")
df_estados.to_parquet(r"Datawarehouse\estados.parquet")